In [1]:
from PIL import Image
import base64
from io import BytesIO
import os
import gradio as gr
from openai import OpenAI
from dotenv import load_dotenv
from pydub import AudioSegment
from pydub.playback import play
from IPython.display import Audio, display
import json
import uuid
import random
import datetime
import time

/Users/saud/Desktop/LLM_Projects/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key and openai_api_key.startswith('sk-proj-'):
    print('API Key exists, good to go')
else:
    print('API Key doesnt exist, troubleshoot the problem!')

API Key exists, good to go


In [3]:
openai = OpenAI()
MODEL = 'gpt-4o-mini'

system_prompt = "You are a helpful assistant for an Airline called FlighAI. "
system_prompt += "Only use the `get_ticket_price` tool to answer ticket price questions like 'How much is a ticket to Paris?'. "
system_prompt += "Only use the `make_flight_booking` tool when the customer wants to book a flight. "
system_prompt += "Never mix the tools. Be short and polite. If unsure, ask a clarifying question."

In [4]:
def get_ticket_price(destination_city):
    ticekt_prices = {'london': '$699', 'berlin': '$599', 'sydney': '$899', 'paris': '$1000', 'dubai': '$399'}
    city = destination_city.lower()
    return ticekt_prices.get(city, 'Unkown')


def make_flight_booking(passenger_name, departure_date, return_date, destination_city, flight_class, airways):
    booking_reference = str(uuid.uuid4())[:6].upper()
    
    available_airways = ['flightai', 'jetsky', 'gofly']
    available_flight_class = ['first', 'business', 'economy']
    
    airways_lower = airways.lower()
    flight_class_lower = flight_class.lower()

    # Validate airways (case-insensitive)
    if airways_lower not in available_airways:
        return f'{airways} is not available. Select from: ["FlightAI", "JetSky", "GoFLY"].'

    # Validate dates
    try:
        dep_date = datetime.strptime(departure_date, "%Y-%m-%d")
        ret_date = datetime.strptime(return_date, "%Y-%m-%d")

        if ret_date <= dep_date:
            return {"error": "Return date must be after the departure date."}
    except ValueError:
        return {"error": "Dates must be in YYYY-MM-DD format."}

    # Validate flight class (case-insensitive)
    if flight_class_lower not in available_flight_class:
        return f'{flight_class} is not available. Select from: ["First", "Business", "Economy"].'

    # Create a booking response
    booking_response = {
        "passenger_name": passenger_name,
        "booking_reference": booking_reference,
        "departure_date": departure_date,
        "return_date": return_date,
        "destination_city": destination_city,
        "flight_class": flight_class.capitalize(),
        "airways": airways.capitalize()        
    }

    return booking_response


In [5]:
price_tool_response_structure = {
    "name": "get_ticket_price",
    "description": "Use this tool ONLY to return the price of a return ticket. Do NOT use it for flight "
    "bookings or collecting booking details.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to.",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

booking_tool_response_structure = {
    "name": "make_flight_booking",
    "description": "Use this tool ONLY when the user says they want to *book* a flight. "
    "Do NOT use this tool if the user just wants to know the ticket price.",
    "parameters": {
        "type": "object",
        "properties": {
            "passenger_name": {
                "type": "string",
                "description": "The name of the customer."
            },
             "departure_date": {
                "type": "string",
                "description": "Date of departure in YYYY-MM-DD format."
            },
             "return_date": {
                "type": "string",
                "description": "Date of return in YYYY-MM-DD format."
            },
             "destination_city": {
                "type": "string",
                "description": "The city that the customer wansts to travel to."
            },
             "flight_class": {
                "type": "string",
                "description": "The class that the customer wants to select for a flight."
            },
             "airways": {
                "type": "string",
                "description": "The name of the airways that the customers wants to travel in."
            },

        },
        "required": ["passenger_name", "departure_date", "return_date", "destination_city", "flight_class", "airways"],
        "additionalProperties": False
    }
}

In [6]:
tools_list = [
    {'type': 'function', 'function': price_tool_response_structure},
    {'type': 'function', 'function': booking_tool_response_structure}
]

In [7]:
def handle_price_tool(assistant_message):
    tool_call = assistant_message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    price = get_ticket_price(city)
    tool_response = {
        'role': 'tool',
        'tool_call_id': tool_call.id,
        'content': json.dumps({'destination_city': city, 'price': price})

    }

    return tool_response, city


def handle_booking_tool(assistant_message):
    tool_call = assistant_message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)

    name = arguments.get('passenger_name')
    dep_date = arguments.get('departure_date')
    ret_date = arguments.get('return_date')
    dest_city = arguments.get('destination_city')
    flight_class = arguments.get('flight_class')
    airways = arguments.get('airways')

    booking_arguments = make_flight_booking(name, dep_date, ret_date, dest_city, flight_class, airways)

    booking_response = {
        'role': 'tool',
        'tool_call_id': tool_call.id,
        'content': json.dumps(booking_arguments)
    }

    return booking_response, dest_city


In [8]:
def artist(city):
    image = openai.images.generate(
        model= 'dall-e-3',
        prompt= f"An image representing a vacation in {city}, showing tourist spots and everything " \
        "unique about Berlin, in a vibrant pop-art style",
        size= '1024x1024',
        n= 1,
        response_format= 'b64_json'
    )
    image_base64 = image.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return Image.open(BytesIO(image_data))

In [9]:
def talker(user_prompt):
    speech = openai.audio.speech.create(
        model= 'tts-1',
        voice= 'nova',
        input= user_prompt
    )
    audio_stream = BytesIO(speech.content)
    output_filename = "output_audio.mp3"
    with open(output_filename, "wb") as f:
        f.write(audio_stream.read())
        
    return output_filename
    
def play_audio(output_filename):
    display(Audio(output_filename, autoplay=True))


In [10]:
def transcribe(audio_file):
    with open(audio_file, 'rb') as audio:    
        response = openai.audio.transcriptions.create(
            model= 'whisper-1',
            file= audio
        )
    return response.text

In [ ]:
def translate_text_to_other_language(text, target_language):    
    prompt = f"Translate the following text to {target_language}:\n\n{text}"
    
    response = openai.chat.completions.create(
        model= MODEL, 
        messages=[
            {"role": "system", "content": "You are a helpful translator. ONLY provide translations. DO NOT add any intro or ending sentences."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content


In [12]:
def airline_customer_support_chatbot(history, language):
    conversation = [{'role': 'system', 'content': system_prompt}] + history 

    response = openai.chat.completions.create(
        model= MODEL,
        messages= conversation, 
        tools= tools_list,
        tool_choice= 'auto',
        temperature= 0.5
    )
    image = None

    if response.choices[0].finish_reason == 'tool_calls':
        assistant_response = response.choices[0].message
        tool_call_name = assistant_response.tool_calls[0].function.name

        if tool_call_name == 'get_ticket_price':
            tool_response, city = handle_price_tool(assistant_response)
        elif tool_call_name == 'make_flight_booking':
            tool_response, city = handle_booking_tool(assistant_response)

        image = artist(city)
        conversation.append(assistant_response)
        conversation.append(tool_response)

        response = openai.chat.completions.create(
            model= MODEL,
            messages= conversation
        )

    result = response.choices[0].message.content
    history += [{'role': 'assistant', 'content': result}]
 
    audio_file = talker(result)
    play_audio(audio_file)
    text = transcribe(audio_file)
    translated_text = translate_text_to_other_language(text, language)

    return translated_text, image, audio_file


In [13]:
def launch_chatbot():
    with gr.Blocks() as demo:
        gr.Markdown("## ✈️ FlighAI Customer Support Chatbot")

        history = gr.State([])

        with gr.Row():
            # Left: Conversation Box
            chat_display = gr.Chatbot(label="Conversation", height=500)

            # Right: Image Preview with same height
            output_image = gr.Image(label="🖼️ Destination Preview", height=500)

        # Second Row: Audio + Translation below
        with gr.Row():
            with gr.Column():
                gr.Markdown("### 🎧 Assistant Audio Response")
                audio_player = gr.Audio(label="", interactive=False)

            with gr.Column():
                gr.Markdown("### 🌍 Translated Response")
                translated_output = gr.Textbox(label="", interactive=False)

        # Input Row at Bottom
        with gr.Row():
            user_input = gr.Textbox(
                placeholder="Type your message and press Enter...",
                label="Your Message",
                lines=1
            )
            language = gr.Dropdown(
                label="Language",
                choices=["English", "Spanish", "French", "German", "Chinese", "Arabic"],
                value="English"
            )

        with gr.Row():
            clear_btn = gr.Button("🧹 Clear Chat")

        # Processing logic
        def user_interaction(message, lang, history_list):
            formatted = []
            for u, b in history_list:
                formatted.append({"role": "user", "content": u})
                formatted.append({"role": "assistant", "content": b})

            bot_text, bot_img, bot_audio = airline_customer_support_chatbot(
                history=formatted,
                language=lang
            )

            new_history = history_list + [(message, bot_text)]
            return "", new_history, bot_img, bot_audio, bot_text, new_history

        user_input.submit(
            fn=user_interaction,
            inputs=[user_input, language, history],
            outputs=[user_input, chat_display, output_image, audio_player, translated_output, history]
        )

        clear_btn.click(lambda: ([], None, None, "", []),
            inputs=None,
            outputs=[chat_display, output_image, audio_player, translated_output, history],
            queue=False
        )

        demo.launch()

In [ ]:
launch_chatbot()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
